# lib_cli_exit_tools — Quickstart

This notebook demonstrates basic usage of `lib_cli_exit_tools` inside a Jupyter environment.

It adjusts `sys.path` to import from the repository `src/` directory so you can run it directly on Binder or locally without installing the package first.


In [ ]:
import sys
import os
import platform
import subprocess
from pathlib import Path
import importlib


# Ensure this notebook can import from the repo's src/ regardless of CWD
def _ensure_src_on_sys_path(pkg: str = "lib_cli_exit_tools") -> str:
    here = Path.cwd()
    # Common locations when notebooks/ is used
    for src in (here / "src", here.parent / "src", here.parent.parent / "src"):
        if (src / pkg).is_dir():
            src = src.resolve()
            if str(src) not in sys.path:
                sys.path.insert(0, str(src))
            return str(src)
    # Fallback: search upwards for a pyproject root
    for p in (here, *here.parents):
        if (p / "pyproject.toml").exists() and (p / "src" / pkg).is_dir():
            src = (p / "src").resolve()
            if str(src) not in sys.path:
                sys.path.insert(0, str(src))
            return str(src)
    raise ModuleNotFoundError(f"Cannot find '{pkg}' under a 'src' directory from {here}. Run 'pip install -e .[dev]' or adjust sys.path manually.")


SRC_DIR = _ensure_src_on_sys_path()
ROOT_DIR = str((Path(SRC_DIR)).parent)

try:
    m = importlib.import_module("lib_cli_exit_tools")
except ModuleNotFoundError:
    print("[setup] Installing lib_cli_exit_tools into this kernel (editable)")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", ROOT_DIR])
    m = importlib.import_module("lib_cli_exit_tools")
get_system_exit_code = m.get_system_exit_code
print_exception_message = m.print_exception_message
config = m.config

print("Python:", platform.python_version())
print("Import OK; traceback default:", config.traceback)

## Try CLI commands
This cell attempts to run the CLI via module (`python -m`) and, if available, all console script aliases.

If the console scripts are not on PATH, set `DO_INSTALL = True` to install the package in-place inside the notebook kernel (requires network).


In [ ]:
import shutil
import subprocess


def run_cmd(cmd: list[str], *, use_src: bool = True) -> int:
    print("$", " ".join(cmd))
    env = os.environ.copy()
    if use_src and "SRC_DIR" in globals():
        env["PYTHONPATH"] = globals()["SRC_DIR"] + os.pathsep + env.get("PYTHONPATH", "")
    try:
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
        print(proc.stdout)
        print(f"[exit] {proc.returncode}")
        return proc.returncode
    except FileNotFoundError:
        print("[skip] command not found")
        return 127


DO_INSTALL = True  # auto-install in this kernel for CLI demo
if DO_INSTALL:
    run_cmd([sys.executable, "-m", "pip", "install", "-e", ROOT_DIR])

# Always try module entry
run_cmd([sys.executable, "-m", "lib_cli_exit_tools", "--help"])
run_cmd([sys.executable, "-m", "lib_cli_exit_tools", "info"])
run_cmd([sys.executable, "-m", "lib_cli_exit_tools", "--version"])

# Try console script aliases if present on PATH
for name in ("lib_cli_exit_tools", "cli-exit-tools", "lib-cli-exit-tools"):
    if shutil.which(name):
        run_cmd([name, "--help"])
    else:
        print(f"[skip] {name} not on PATH; set DO_INSTALL=True to enable")

## Map exceptions to exit codes
Use `get_system_exit_code(e)` to map a Python exception to an OS-friendly exit code.


In [ ]:
try:
    raise FileNotFoundError("missing.txt")
except Exception as e:
    code = get_system_exit_code(e)
    print("Mapped exit code:", code)

## Print concise error messages
Toggle `config.traceback` to include or hide tracebacks in error output.


In [ ]:
config.traceback = False
try:
    raise PermissionError("no access")
except Exception as _:
    # Emits a concise one-line message to stdout/stderr (depending on implementation)
    print_exception_message()

## Notes
- In applications, call `raise SystemExit(get_system_exit_code(e))` to terminate with the mapped exit code.
- For CLI usage, see `lib_cli_exit_tools --help` outside the notebook.
